In [208]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
import numpy.matlib
import random
import os
import pathlib
import time

## Classes

In [209]:
class Individual:
  def __init__(self,c, w, fitness=math.inf):
    self.c = c
    self.w = w
    self.fitness = fitness

  # Getters
  def getFitness(self):
    return self.fitness

  # Setters
  def setFitness(self,f):
    self.fitness = f
  
  # Methods
  def printIndividual(self):
    return f'C (size): {self.c.shape}, W (size): {self.w.shape}, fitness: {self.fitness}\nC: {self.c}\nW: {self.w}'.format(self=self)


In [210]:
class Particle(Individual):
  def __init__(self,c, w, fitness=math.inf):
    Individual.__init__(self, c, w, fitness)
    self.best_fitness = fitness
    self.best_w_position = w
    self.best_c_position = c
    self.actual_w_position = w
    self.actual_c_position = c
    self.actual_w_velocity = np.zeros(w.shape) # All swarm's weight velocities are set to 0 at the start
    self.actual_c_velocity = np.zeros(c.shape) # All swarm's chromosome velocities are set to 0 at the start

  # Methods
  def updateChromosomeVelocity(self, swarm, w:float, c1:float, r1:float, c2:float, r2:float):
    for k in range(len(self.actual_c_velocity)):
      self.actual_c_velocity[k] = ( 
                                    (w * self.actual_c_velocity[k]) +
                                    (c1*r1*(self.best_c_position[k] - self.actual_c_position[k])) +
                                    (c2*r2*(swarm.global_best_c[k] - self.actual_c_position[k]))
                                  )
    return self.actual_c_velocity
                      
  def updateWeightsVelocity(self, swarm, w:float, c1:float, r1:float, c2:float, r2:float):
    for k in range(len(self.actual_w_velocity)):
      self.actual_w_velocity[k] = ( 
                                    (w * self.actual_w_velocity[k]) +
                                    (c1*r1*(self.best_w_position[k] - self.actual_w_position[k])) +
                                    (c2*r2*(swarm.global_best_w[k] - self.actual_w_position[k]))
                                  )
    return self.actual_w_velocity
  '''
  def updateWeightsPosition(self, new_W):
    self.wPosition = new_W
    
  def updateChromosomePosition(self, new_C):
    self.cPosition = new_C

  def updateWeightsVelocity(self, new_W):
    self.actual_w_velocity = new_W

  def updatebest_fitness(self,new_fitness):
    self.best_fitness = new_fitness
  '''

In [211]:
class Population:
  def __init__(self,size):
    self.size = size
    self.gene_list = []
    self.fitness_list = []
    self.best_gene = None

  # Methods
  def printPopulation(self):
    print(f'Population size: {len(self.gene_list)}, best fitness: {self.best_gene.fitness}')
  def addGeneToList(self,gen):
    self.gene_list.append(gen)

  def addFitnessToList(self,fitness):
    self.fitness_list.append(fitness)

  def insertbest_gene(self,gen):
    self.best_gene = gen
    self.gene_list.append(gen)
    self.fitness_list.append(gen.fitness)

  def updatebest_gene(self, gen):
    self.best_gene = gen

  def createPopulation(self,K,D):
    for i in range(self.size):
      gen = Individual(random_C_generator(K), random_W_generator(K,D))
      self.gene_list.append(gen)
    
    # Once the population is set, we choose a random best gene
    random_index = np.random.randint(low=0, high=self.size)
    self.best_gene = self.gene_list[random_index]
    return self

In [212]:
class Swarm(Population):
  def __init__(self,size):
    Population.__init__(self, size)
    self.global_best_w = None
    self.global_best_c = None

  # Getters
  def getBestSwarmWeights(self):
    return self.global_best_w

  def getBestSwarmChromosome(self):
    return self.global_best_c

  # Setters
  def setBestSwarmWeigths(self,newW):
    self.global_best_w = newW

  def setBestSwarmChromosome(self,newC):
    self.global_best_c = newC

  def initializeSwarm(self):
    for i in range(self.size):
      p = Particle(random_C_generator(K), random_W_generator(K,optimalD))
      self.gene_list.append(p)
          
    # Once the swarm is set, we choose a random best gene
    randomIndex = np.random.randint(low=0, high=self.size)
    self.best_gene = self.gene_list[randomIndex]
    self.global_best_c = self.best_gene.c
    self.global_best_w = self.best_gene.w  
    return self

In [213]:
class Reef:
  def __init__(self,rho0, R):
    self.reefSize = R
    self.freeOccupiedRate = rho0
    self.gene_list = np.full([R], None)
    self.fitness_list = []
    self.best_gene = None

  # Methods
  def addGeneToList(self,gen):
    self.gene_list.append(gen);

  def addFitnessToList(self,fitness):
    self.fitness_list.append(fitness);

  def insertbest_gene(self,gen):
    self.best_gene = gen
    self.gene_list.append(gen)
    self.fitness_list.append(gen.fitness)

  def initializeReef(self, K, D):
    occupiedHoles = int(self.reefSize * self.freeOccupiedRate) # Partially Occupied
    print("Occupied holes in reef", occupiedHoles)
    for i in range(occupiedHoles): 
      # Choose random hole
      random_hole_index = np.random.randint(low=0, high=self.reefSize)
      
      while 1: # If that hole is occupied then choose again until it finds an empty one 
        if self.gene_list[random_hole_index] == None:
          break
        random_hole_index = np.random.randint(low=0, high=self.reefSize)
        '''
        if not self.gene_list[random_hole_index][0]:
          break
        else:
          random_hole_index = np.random.randint(0,self.gene_list.shape[0])
        '''

      # Insert larvae in that hole
      larvae = Individual(random_C_generator(K), random_W_generator(K,D))
      self.gene_list[random_hole_index] = larvae
      
      # Once the reef is set, we choose a random best gene
      random_best_hole_index = np.random.randint(0,self.reefSize)
      
      while 1: # If that hole is occupied then choose again until it finds an empty one 
        if not self.gene_list[random_best_hole_index]:
          self.best_gene = self.gene_list[random_best_hole_index]
          break
        else:
          random_best_hole_index = np.random.randint(0,self.gene_list.shape)

    return self

## Functions

### NumPy custom functions

In [214]:
def sumMatrixes(mat1,mat2):
  return np.add(mat1,np.asarray(mat2))

def subtractMatrixes(mat1,mat2):
  return np.subtract(mat1,np.asarray(mat2))

def dotMultiply(mat1,mat2):
  return np.multiply(mat1,np.asarray(mat2))

def multiplyMatrixes(mat1,mat2):
  return np.matmul(mat1,np.asarray(mat2))

def dotDivide(mat1,mat2):
  return np.divide(mat1,np.asarray(mat2))

### Population

In [215]:
def random_C_generator(K):
  # randint returns from: inclusive -[low,high)- exclusive.
  return np.random.randint(low= 0, high= 1 + 1, size=[K])

def random_W_generator(K,D):
  return np.random.uniform(low= -1, high= 1, size=[K,D])

In [216]:
def createIndividual(K,D):
  return Individual(random_C_generator(K), random_W_generator(K,D))


#### Reproduction

In [217]:
def InternalReproduction(gen):
  # Mutate chromosome 
  mutate_point = np.random.randint(0, len(gen.c)) # Choose a random point to mutate
  gen.c[mutate_point] = 1 - gen.c[mutate_point] # Mutate the point
  
  # Mutate weights
  nRows = gen.w.shape[0] 
  nColumns = gen.w.shape[1] 
  mutate_column = np.random.randint(0, nColumns) # Choose a random column to mutate
  
  gen.w[:,mutate_column] = np.random.uniform(-1,1,nRows) # Mutate the column

In [218]:
def ExternalReproduction(father,mother):
  crossover_c_point = np.random.randint(0, len(father.c))
  crossover_column_w_point = random.randint(0, len(father.w[0]))
  
  # Offsprings
  offspring1 = Individual( c = np.concatenate((father.c[:crossover_c_point], mother.c[crossover_c_point:])), 
                           w = np.concatenate((father.w[:,:crossover_column_w_point], mother.w[:,crossover_column_w_point:]), axis=1) ) 
  
  offspring2 = Individual( c = np.concatenate((mother.c[:crossover_c_point], father.c[crossover_c_point:])), 
                           w = np.concatenate((mother.w[:,:crossover_column_w_point], father.w[:,crossover_column_w_point:]), axis=1) ) 
                       
  return [offspring1,offspring2] 

In [219]:
'''     REPRODUCTION TESTS PASSED     '''

'''
populationTest = Population(size=10).createPopulation(K=4,D=3)

hijosTest = ExternalReproduction(populationTest.gene_list[0],populationTest.gene_list[1])

for index, i in enumerate(hijosTest):
    print(f"Son:{index}{i.c}")

print("Before:",populationTest.gene_list[0].c)
print("Before:",populationTest.gene_list[0].w)
InternalReproduction(populationTest.gene_list[0])

print("After:",populationTest.gene_list[0].c)
print("After:",populationTest.gene_list[0].w)
'''

'\npopulationTest = Population(size=10).createPopulation(K=4,D=3)\n\nhijosTest = ExternalReproduction(populationTest.gene_list[0],populationTest.gene_list[1])\n\nfor index, i in enumerate(hijosTest):\n    print(f"Son:{index}{i.c}")\n\nprint("Before:",populationTest.gene_list[0].c)\nprint("Before:",populationTest.gene_list[0].w)\nInternalReproduction(populationTest.gene_list[0])\n\nprint("After:",populationTest.gene_list[0].c)\nprint("After:",populationTest.gene_list[0].w)\n'

In [220]:
def RouletteWheelSelection(population): 
  # Create a population for the chosen individuals of the Roulette
  roulettePopulation = Population(population.size)

  # Make sure we don't lose our best gene in the roulette 
  roulettePopulation.insertbest_gene(population.best_gene)

  # Total population fitness (S)
  S = sum([chromosome.fitness for chromosome in population.gene_list])
  
  # Population chromosomes' relative probabilities
  rel_prob = [chromosome.fitness/S for chromosome in population.gene_list]

  for i in range(population.size - 1):
    # Generate a random uniform number (r) - (0,1]
    r = np.random.uniform() 
    # Find the first index for which q_i < r
    for index,individual in enumerate(population.gene_list): 
      r -= rel_prob[index]
      if r < 0: 
        roulettePopulation.addGeneToList(individual) 
        roulettePopulation.addFitnessToList(individual.fitness) 
        break

  return roulettePopulation  

In [221]:
# This functions reproduces a population.
# It crossover the parents (external reproduction) and mutate the offspring (internal reproduction).
# Returns the input population, updated with the crossover and mutated childs

def ReproducePopulation(population):
  childs = [] # list to store future offsprings
  crossoverProb = np.random.uniform() # crossover probability
  mutationProb = np.random.uniform() # mutation probability
  gene_listSize = len(population.gene_list)

  for index,individual in enumerate(population.gene_list):
    # Generate a random number and check if it's over the crossover probability
    if(random.random() >= crossoverProb):
      random_mother_index = np.random.randint(0,gene_listSize)

      while(random_mother_index == index): # Make sure that the mother is a different individual in the population
        random_mother_index = np.random.randint(0,gene_listSize)

      mother = population.gene_list[random_mother_index]

      # Crossover parents
      offsprings = ExternalReproduction(individual, mother)

      # Mutate offsprings
      if(random.random() < mutationProb): 
        InternalReproduction(offsprings[0])

      if(random.random() < mutationProb):
        InternalReproduction(offsprings[1])

      childs.append(offsprings[0])
      childs.append(offsprings[1])
  
  # Update population's Genes list with newly-created childs
  population.gene_list = np.concatenate((population.gene_list, childs))


  return population

In [222]:
'''    FINAL REPRODUCTION FUNCTION TESTS PASSED     '''
'''
populationTest = Population(size=10).createPopulation(K=9,D=2)

# Print population before reproduction
populationTest.printPopulation()

# Reproduce population
populationReproduced = ReproducePopulation(populationTest)

# Print population before reproduction
populationReproduced.printPopulation()
'''

'\npopulationTest = Population(size=10).createPopulation(K=9,D=2)\n\n# Print population before reproduction\npopulationTest.printPopulation()\n\n# Reproduce population\npopulationReproduced = ReproducePopulation(populationTest)\n\n# Print population before reproduction\npopulationReproduced.printPopulation()\n'

#### Swarm

In [223]:
def UpdateVelocitiesAndPositionsPSO(swarm: Swarm, w:float, c1:float, c2:float):
  for index, p in enumerate(swarm.gene_list): # For each particle in the swarm

    # Velocities
    r1 = np.random.uniform()
    r2 = np.random.uniform()

    # Chromosome loop
    for k in range(len(p.c)):
      # Print all parameters in next equation 
      p.actual_c_velocity[k] = ( 
                                (w * p.actual_c_velocity[k]) +
                                (c1*r1*(p.best_c_position[k] - p.actual_c_position[k])) +
                                (c2*r2*(swarm.global_best_c[k] - p.actual_c_position[k]))
                               )
      
      #print(f"Particle {index}  actual Velocity {k}: {p.actual_c_velocity[k]}")

      velocityProbability = 2 / math.pi * math.atan((math.pi*0.5)*p.actual_c_velocity[k]) #|2⁄𝜋 × arctan ((𝜋 2) × 𝑉𝑡+1
      #print(f"Particle {index} velocityProbability {k}: {velocityProbability}")

      # Update to next position
      if(np.random.uniform() < velocityProbability):
        p.actual_c_position[k] += 1
      else:
        p.actual_c_position[k] -= 0
    # end chromosome loop

    # Weight loop
    for i in range((p.w).shape[0]):
      for j in range((p.w).shape[1]):
        p.actual_w_velocity[i,j] = ( 
                                    (w * p.actual_w_velocity[i,j]) +
                                    (c1*r1*(p.best_w_position[i,j] - p.actual_w_position[i,j])) +
                                    (c2*r2*(swarm.global_best_w[i,j] - p.actual_w_position[i,j]))
                                   )

        # Update to next position
        p.actual_w_position[i,j] += p.actual_w_velocity[i,j]
    # end weight loop
  # end particle loop
  return swarm

In [224]:
matrix = np.array([[1,2,3],[4,5,6],[7,8,9]])

print(matrix[0,2])
print(matrix[0][2])

for i in range(matrix.shape[0]):
    print("jump")
    for j in range(matrix.shape[1]):
        print(matrix[i,j])

3
3
jump
1
2
3
jump
4
5
6
jump
7
8
9


## Data Processing

In [225]:
CSV_NAME = "car-evaluation.csv"
CSV_PATH = "E:/Perfil/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/" + CSV_NAME
df = pd.read_csv(CSV_PATH, sep=" ", header=None)

end = df.shape[1]
X = df.iloc[:, :end-1] # iloc[rows,[cols_start,cols_end)] <- Dataframe object
Y = df.iloc[:, end-1] # iloc[rows,col_index] <- Dataframe series

# IMPORTANT, otherwise we would be using DataFrames and matrix operations won't work
X = X.to_numpy()
Y = Y.to_numpy()
J = len(np.unique(Y))

N,K = X.shape[0],X.shape[1] 

In [226]:
# Scaling X (min-max normalization)
X_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

# First partition training/test
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42) # 20% test data

# Second partition validation
X_trainVal, X_testVal, Y_trainVal, Y_testVal = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # 20% validation data

## ELM

In [227]:
# Outputs the fitness for a given Gene
def ComputeChromosomeFitness(individual, D,C,trainingX, trainingY, testX, testY):
    # W
    individualWeights = individual.w

    # Feature-selection Bias given Gene's chromosome
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiplyMatrixes(trainingX, individualWeights) + BiasTrainingMatrix
    activationTest = multiplyMatrixes(testX, individualWeights) + BiasTestMatrix
    
    H_Training = dotDivide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dotDivide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiplyMatrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dotDivide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiplyMatrixes(inverse, np.transpose(H_Training))
    Beta = multiplyMatrixes(Beta, trainingY)

    # Output
    Y_predicted = multiplyMatrixes(H_Test, Beta)
    fitness = np.linalg.norm(Y_predicted - testY)

    return fitness

In [228]:
# Updates the fitnesses of a given population of Genes and also, set the new best Gene 
def ComputePopulationFitness(population, D,C,trainingX, trainingY, testX, testY):
  # Clear previous fitnesses
  population.fitness_list.clear()

  for index,individual in enumerate(population.gene_list):
    f_i = ComputeChromosomeFitness(individual, D, C, trainingX, trainingY, testX, testY)
     
      # Set fitness
    individual.setFitness(f_i)
    #print("Best Gene Fi middle:", population.best_gene.fitness)
    population.addFitnessToList(f_i)

      # Update Best Gene in Population
    if(individual.fitness < population.best_gene.fitness):
      population.best_gene = individual

In [229]:
# Updates the fitnesses of a given population of Genes and also, set the new best Gene 
def ComputeSwarmFitness(swarm: Swarm, D,C,trainingX, trainingY, testX, testY):
  # Clear previous fitnesses
  swarm.fitness_list.clear()

  for index,particle in enumerate(swarm.gene_list):    
    f_i = ComputeChromosomeFitness(particle, D, C, trainingX, trainingY, testX, testY)

    # Set particle's fitness
    particle.setFitness(f_i)
    swarm.addFitnessToList(f_i)

    # Update Pb_i for each particle and find the best gene 
    if particle.fitness < particle.best_fitness:
      particle.best_fitness = particle.fitness
      
      if particle.fitness < swarm.best_gene.fitness:
        swarm.best_gene = particle

  # Update Gbest of the swarm
  swarm.setBestSwarmChromosome(swarm.best_gene.c)
  swarm.setBestSwarmWeigths(swarm.best_gene.w)
  print("Global best fitness: ", swarm.best_gene.fitness)

In [230]:
def TrainModelAndOutputResults(weigths, D,C,trainingX, trainingY, testX, testY):
      # W
    W = weigths 

    # Bias
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiplyMatrixes(trainingX, W) + BiasTrainingMatrix
    activationTest = multiplyMatrixes(testX, W) + BiasTestMatrix
    
    H_Training = dotDivide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dotDivide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiplyMatrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dotDivide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiplyMatrixes(inverse, np.transpose(H_Training))
    Beta = multiplyMatrixes(Beta, trainingY)

    # Prediction
    predictedLabels = multiplyMatrixes(H_Test, Beta)

    correctPrediction = 0
    for i in range(testY.shape[0]):
      if np.round(predictedLabels[i]) == testY[i]:
        correctPrediction +=1
    
    CCR = correctPrediction / testY.shape[0]
    return CCR * 100

## Genetic Algorithm

In [231]:
def GeneticAlgorithm(gaPopulation, nGenerations, optimalD, optimalC, trainX, trainY, testX, testY):
  t = 0
  while t < nGenerations:
      # Reproduction
    ReproducePopulation(gaPopulation)  
      # Compute Fitness
    ComputePopulationFitness(gaPopulation, optimalD, optimalC, trainX, trainY, testX, testY)

    print("Generation: ", t, " Best Fitness: ", gaPopulation.best_gene.fitness)
      # Selector operator
    gaPopulation = RouletteWheelSelection(gaPopulation)
      # Continue iterating
    t += 1
  
  return gaPopulation

In [232]:
  # GA
  '''
  sizePopulation = 1
  nGenerations = 1
  crossover_prob = 0.65 # This value and mutation_prob are randomly set at Reproduce Population functino
  mutation_prob = 0.15
  optimalD = 1000
  optimalC = 1

  gaStartExTime = time.time()
  print("- - - - - - - -")
  print("Starting GA ...")
  
  # Init population  
  gaFirstGeneration = Population(sizePopulation).createPopulation(K,optimalD)

  # Apply Algorithm and Evolve
  lastGeneration = GeneticAlgorithm(gaFirstGeneration, nGenerations, optimalD, optimalC, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
  print("Best fitness:", lastGeneration.best_gene.fitness)

  # Print lastGeneration best gene
  print("Best feature selection: ", lastGeneration.best_gene.c)
  print("Best Weights: ", lastGeneration.best_gene.w)
  # Train ELM with best W
  GA_CCR = TrainModelAndOutputResults(lastGeneration.best_gene.w, optimalD, optimalC, X_train, Y_train, X_test, Y_test)
  
  print("GA CCR:", GA_CCR, "\nError:", 100-GA_CCR)
  print("GA execution time: ", time.time() - gaStartExTime, "sec")
  print("- - - - - - - -")
  '''

'\nsizePopulation = 1\nnGenerations = 1\ncrossover_prob = 0.65 # This value and mutation_prob are randomly set at Reproduce Population functino\nmutation_prob = 0.15\noptimalD = 1000\noptimalC = 1\n\ngaStartExTime = time.time()\nprint("- - - - - - - -")\nprint("Starting GA ...")\n\n# Init population  \ngaFirstGeneration = Population(sizePopulation).createPopulation(K,optimalD)\n\n# Apply Algorithm and Evolve\nlastGeneration = GeneticAlgorithm(gaFirstGeneration, nGenerations, optimalD, optimalC, X_trainVal, Y_trainVal, X_testVal, Y_testVal)\nprint("Best fitness:", lastGeneration.best_gene.fitness)\n\n# Print lastGeneration best gene\nprint("Best feature selection: ", lastGeneration.best_gene.c)\nprint("Best Weights: ", lastGeneration.best_gene.w)\n# Train ELM with best W\nGA_CCR = TrainModelAndOutputResults(lastGeneration.best_gene.w, optimalD, optimalC, X_train, Y_train, X_test, Y_test)\n\nprint("GA CCR:", GA_CCR, "\nError:", 100-GA_CCR)\nprint("GA execution time: ", time.time() - gaSt

## Particle Swarm Optimization

In [233]:
def PSO_Algorithm(swarm, maxGenerations, w, c1, c2, dampingW, optimalD, optimalC, trainX, trainY, testX, testY):
  t = 0
  while t < maxGenerations:
    # Evaluate Particles' Fitness and update Pb_i and Gb_i
    ComputeSwarmFitness(swarm, optimalD, optimalC, trainX, trainY, testX, testY)
      
    # Update Velocity and Position of each particle
    UpdateVelocitiesAndPositionsPSO(swarm, w, c1, c2)

    # Damping inertia
    w = w * dampingW

    # Increase step
    t += 1

  return 1

In [234]:
# PSO
'''
swarm_size = 2 # total population
num_generations = 1
w = 1 # inertia weight coefficient
dampingW = 0.99 # damping for inertia coefficient
c1 = 2 # cognitive coefficient
c2 = 2 # social coefficient
optimalD = 1000
optimalC = 1

print("Starting PSO method...")
pso_start_execution_time = time.time()
# Init population
swarm = Swarm(swarm_size).initializeSwarm() # Swarm velocity is set to 0 by default
# Apply Algorithm and Iterate
PSO_Algorithm(swarm, num_generations, w, c1, c2, dampingW, optimalD, optimalC, X_trainVal, Y_trainVal, X_testVal, Y_testVal)

print("PSO Completed")
# Train ELM with best W
PSO_CCR = TrainModelAndOutputResults(swarm.global_best_w, optimalD, optimalC, X_train, Y_train, X_test, Y_test)

print("PSO CCR:", PSO_CCR, "\nError:", 100-PSO_CCR)
print("PSO execution time: ", time.time() - pso_start_execution_time, "sec")

# Print best solution found
print("Best feature selection: ", swarm.global_best_c)
print("Best Weights: ", swarm.global_best_w)
'''

'\nswarm_size = 2 # total population\nnum_generations = 1\nw = 1 # inertia weight coefficient\ndampingW = 0.99 # damping for inertia coefficient\nc1 = 2 # cognitive coefficient\nc2 = 2 # social coefficient\noptimalD = 1000\noptimalC = 1\n\nprint("Starting PSO method...")\npso_start_execution_time = time.time()\n# Init population\nswarm = Swarm(swarm_size).initializeSwarm() # Swarm velocity is set to 0 by default\n# Apply Algorithm and Iterate\nPSO_Algorithm(swarm, num_generations, w, c1, c2, dampingW, optimalD, optimalC, X_trainVal, Y_trainVal, X_testVal, Y_testVal)\n\nprint("PSO Completed")\n# Train ELM with best W\nPSO_CCR = TrainModelAndOutputResults(swarm.global_best_w, optimalD, optimalC, X_train, Y_train, X_test, Y_test)\n\nprint("PSO CCR:", PSO_CCR, "\nError:", 100-PSO_CCR)\nprint("PSO execution time: ", time.time() - pso_start_execution_time, "sec")\n\n# Print best solution found\nprint("Best feature selection: ", swarm.global_best_c)\nprint("Best Weights: ", swarm.global_best_

## Coral Reef Optimization

In [235]:
def CROAlgorithm():
  t = 0
  return 1
  '''
  while t < coralGenerations:
    # Update variables
    F_broadcast = np.random.uniform() # broadcast fraction
    F_brooding = 1 - F_broadcast # brooding fraction
    Pd = 0.2 # predation probability
    
    # Broadcast and Brooding
    
    # Asexual reproduction

    # Predation 

    # Fitness Evaluation

    # Increase step
    t += 1
    # - end WHILE

  print("Best CRO: ", "coral X")
  '''

In [236]:
# CRO
reefSize = 10
rho0 = 0.6 # free/occupied rate
eta = 2 # larva's attempts
coralGenerations = 10 
F_broadcast = np.random.uniform() # broadcast fraction
F_brooding = 1 - F_broadcast # brooding fraction
Pd = 0.2 # predation probability
optimalD = 1000
optimalC = 1

croStartExTime = time.time()
print("- - - - - - - -")
print("Starting CRO method...")
# Init population
reefFirstGeneration = Reef(rho0,reefSize).initializeReef(K,optimalD)

# Print reef
print("Reef: ", reefFirstGeneration.gene_list)
print("Best gene: ", reefFirstGeneration.best_gene)
for index, l in enumerate(reefFirstGeneration.gene_list):
    if not l == None:
        print(f"Larvae {index}: C: {l.c} \n W: {l.w}")

        print("- - - - - - - \n")
# Apply Algorithm and Evolve
#CROAlgorithm(swarm, iterations, w, c1, c2, dampingW, optimalD, optimalC, X_trainVal, y_trainVal, X_testVal, y_testVal)

print("CRO Completed")

print("CRO execution time: ", time.time() - croStartExTime, "sec")
print("- - - - - - - -")

- - - - - - - -
Starting CRO method...
Occupied holes in reef 6
Reef:  [<__main__.Individual object at 0x00000174EA3BDE50>
 <__main__.Individual object at 0x00000174EA091730> None
 <__main__.Individual object at 0x00000174EA091880> None
 <__main__.Individual object at 0x00000174EA091FD0> None None
Best gene:  None
Larvae 0: C: [0 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0 0 1 1] 
 W: [[ 0.41363849  0.6819452  -0.26502789 ... -0.71336069  0.34788952
  -0.77637698]
 [ 0.3898508  -0.40519146  0.64093111 ... -0.38965457 -0.7395827
  -0.4673808 ]
 [-0.47739104  0.27647957 -0.08585409 ...  0.45711518 -0.38479759
  -0.23248996]
 ...
 [-0.60453097  0.9068808  -0.22385051 ... -0.6612093   0.77369628
   0.76724646]
 [ 0.70277661 -0.61772076 -0.62446861 ...  0.19024721 -0.43193272
  -0.31764958]
 [-0.67700283 -0.12350485 -0.40429748 ... -0.24299837 -0.6079102
  -0.43382661]]
- - - - - - - 

Larvae 1: C: [0 0 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0] 
 W: [[-0.23424358 -0.37602054 -0.4820093  ...  0.29569136 

# MAIN

In [237]:
if __name__ == '__main__':
  # ! We should set random number generator seed
  #np.random.seed(seed=0)

  # - - ALGORITHMS - - -
  print("Hello guys...")


Hello guys...


In [238]:
# Create an Individual 
debugIndividual = createIndividual(K=21,D=100)

print(debugIndividual.w.shape)

skere = np.zeros(shape=debugIndividual.w.shape)

print(skere.shape)

(21, 100)
(21, 100)
